In [61]:
#Importation des differentes packages
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import widgets
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import re
df = pd.read_csv("1335.vdb.tab", sep ='\t')



In [62]:
pattern='\((\d+)\)' #regex to target numbers enclosed in parenthesis
sub_dtf=df['OTUConTaxonomy'].map(lambda x: re.sub(pattern, '', x).rstrip(";")).str.split(";",expand= True)
df[['regne','embranchement','classe','ordre', 'famille', 'genre']] = sub_dtf
df

,OTUNumber,OTUConTaxonomy,door_in_1,door_in_2,faucet_handle_1,faucet_handle_2,sink_floor_1,sink_floor_2,soap_dispenser_1,stall_in_1,...,toilet_flush_handle_1,toilet_flush_handle_2,toilet_seat_1,toilet_seat_2,regne,embranchement,classe,ordre,famille,genre
0,Otu00001,Bacteria(100);Actinobacteria(100);Actinobacter...,1381,1032,335,577,38,36,1115,598,...,13,329,76,156,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Propionibacteriaceae,Propionibacterium
1,Otu00002,Bacteria(100);Proteobacteria(100);Gammaproteob...,20,17,1686,3,411,564,128,9,...,0,25,258,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Enhydrobacter
2,Otu00003,Bacteria(100);Actinobacteria(100);Actinobacter...,8,47,6,418,15,8,84,45,...,3,283,41,1059,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Corynebacteriaceae,Corynebacterium
3,Otu00004,Bacteria(100);Actinobacteria(100);Actinobacter...,6,27,5,6,452,275,63,0,...,9,51,5,3,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Micrococcaceae,Arthrobacter
4,Otu00005,Bacteria(100);Actinobacteria(100);Actinobacter...,0,29,0,648,2,8,3,32,...,0,206,7,95,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Corynebacteriaceae,Corynebacterium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4100,Otu04101,Bacteria(100);Bacteroidetes(100);Cytophagia(10...,0,0,0,0,0,1,0,0,...,0,0,0,0,Bacteria,Bacteroidetes,Cytophagia,Cytophagales,Cytophagaceae,Hymenobacter
4101,Otu04102,Bacteria(100);Proteobacteria(100);Alphaproteob...,0,0,0,0,0,0,0,0,...,1,0,0,0,Bacteria,Proteobacteria,Alphaproteobacteria,Rhizobiales,unclassified,unclassified
4102,Otu04103,Bacteria(100);Firmicutes(100);unclassified(100...,0,0,0,0,0,0,1,0,...,0,0,0,0,Bacteria,Firmicutes,unclassified,unclassified,unclassified,unclassified
4103,Otu04104,Bacteria(100);Acidobacteria(100);Acidobacteria...,0,0,0,0,0,1,0,0,...,0,0,0,0,Bacteria,Acidobacteria,Acidobacteria_Gp6,Gp6,unclassified,unclassified


In [63]:
###ACP
# ACP des differentes echantillons 

df['name']= df['famille']+' '+df['genre'] 

df = df[df[ 'name']  != 'unclassified unclassified']

matrice=df.groupby(by="name").sum()


samples = list(matrice.columns.values)


#normalisation de la matrice transposé regrouper par embranchement 
matriceT = matrice.transpose()
matriceTN = StandardScaler().fit_transform(matriceT)

#Calcul de l'ACP avec sklearn en conservant 2 var
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(matriceTN)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

principalDf['sample']=samples

fig = px.scatter(principalDf, x='principal component 1', y='principal component 2', color='sample', 
                 title="ACP sur les echantilons et les bacteries presentes")
fig.show()

In [64]:
###build group
group1=["toilet_floor_1","toilet_floor_2","sink_floor_1","sink_floor_2"]
group2=["door_in_1","door_in_2","toilet_seat_1","toilet_seat_2","stall_in_1","soap_dispenser_1",
        "faucet_handle_2","faucet_handle_1","toilet_flush_handle_2"]
group3=["toilet_flush_handle_1"]

df_grp=pd.DataFrame()
df_grp[["regne","embranchement","classe","ordre","famille","genre","name"]]=df[["regne","embranchement","classe","ordre","famille","genre","name"]]
df_grp['group1'] = df[group1].mean(axis=1)
df_grp['group2'] = df[group2].mean(axis=1)
df_grp['group3'] = df[group3].mean(axis=1)
df_grp.sum() ###N.B : SOMME NON IDENTIQUE ON PASSE EN ABONDANCE RELATIVE !!
matrice=df_grp.groupby(by="name").sum()
counts = matrice.div(matrice.sum(axis=0))*100 ### % d'abondance relative
counts.sum(axis=1)
print(matrice)
counts.sum(axis=1)

                                    group1     group2  group3
name                                                         
Acetobacteraceae Acetobacter          1.50   4.000000     0.0
Acetobacteraceae Belnapia             0.50   0.000000     5.0
Acetobacteraceae Endobacter           0.00   0.555556     0.0
Acetobacteraceae Roseomonas          69.50  28.000000     6.0
Acetobacteraceae Stella               0.25   0.000000     0.0
...                                    ...        ...     ...
Xanthomonadaceae Pseudoxanthomonas    3.50   1.333333     5.0
Xanthomonadaceae Stenotrophomonas     3.50   6.666667     1.0
Xanthomonadaceae Thermomonas          2.50   0.777778    81.0
Xanthomonadaceae Xanthomonas          0.50   0.666667     0.0
Xanthomonadaceae unclassified         5.25   5.333333    66.0

[422 rows x 3 columns]


name
Acetobacteraceae Acetobacter          0.192928
Acetobacteraceae Belnapia             0.268904
Acetobacteraceae Endobacter           0.019075
Acetobacteraceae Roseomonas           3.837347
Acetobacteraceae Stella               0.009264
                                        ...   
Xanthomonadaceae Pseudoxanthomonas    0.425858
Xanthomonadaceae Stenotrophomonas     0.408680
Xanthomonadaceae Thermomonas          4.175434
Xanthomonadaceae Xanthomonas          0.041419
Xanthomonadaceae unclassified         3.682632
Length: 422, dtype: float64

In [65]:
import numpy as np
###convert faible abondance to other 
cut_off=2 #en %
test_df= counts.loc[lambda df: (counts['group1'] > cut_off) | (counts['group2'] > cut_off) | (counts['group3'] > cut_off) ]
#new_row = [100-test_df['group1'].sum(), 100-test_df['group2'].sum(), 100-test_df['group3'].sum()]
#test_df.append(new_row, ignore_index=True)
#new_row
#test_df.loc["new"]
test_df.loc['others'] = [100-test_df['group1'].sum(), 100-test_df['group2'].sum(), 100-test_df['group3'].sum()]
#test_df.summary

/home/arnaud/miniconda3/lib/python3.7/site-packages/pandas-1.0.4-py3.7-linux-x86_64.egg/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [84]:
test_df.shape

(25, 3)

In [85]:
# ACP des different GROUP -> check if group bien distinct
matrice=test_df


samples = list(matrice.columns.values)


#normalisation de la matrice transposé regrouper par embranchement 
matriceT = matrice.transpose()
matriceTN = StandardScaler().fit_transform(matriceT)

#Calcul de l'ACP avec sklearn en conservant 2 var
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(matriceTN)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

principalDf['sample']=samples

fig = px.scatter(principalDf, x='principal component 1', y='principal component 2', color='sample', 
                 title="ACP sur les echantilons et les bacteries presentes")
fig.show()

In [87]:
print(matrice.shape)
embranchement = list(matrice.index.values)
samples = list(matrice.columns.values)
print(matrice.sum())

fig = px.bar(matrice.transpose(), 
            x=samples,
            y=embranchement, 
             color_discrete_sequence=px.colors.sequential.Viridis, 
             title="Répartition des differents embranchements")
fig.show()

(25, 3)
group1    100.0
group2    100.0
group3    100.0
dtype: float64


In [88]:

indexNamesArr = matrice.index.values

embranchement = list(indexNamesArr)
samples = list(matrice.columns.values)


fig = go.Figure(data=go.Heatmap(
                   z=matrice,
                   x=samples,
                   y=embranchement,
                   hoverongaps = False))
fig.show()